In [ ]:
CREATE OR REPLACE TABLE dim_vehicle (
    vehicle_id INT AUTOINCREMENT PRIMARY KEY,
    vin STRING,
    make STRING,
    model STRING,
    year_model INTEGER,
    electric_vehicle_type STRING,
    base_msrp INTEGER
);


In [ ]:
CREATE OR REPLACE TABLE dim_city (
    city_id INT AUTOINCREMENT PRIMARY KEY,
    city_name STRING,
    state STRING,
    county STRING,
    postal_code STRING
);


In [ ]:
CREATE OR REPLACE TABLE dim_time (
    time_id INT AUTOINCREMENT PRIMARY KEY,
    sale_date TIMESTAMP,
    sale_day INTEGER,
    sale_month INTEGER,
    sale_quarter INTEGER,
    sale_year INTEGER,
    resale_date TIMESTAMP
);


In [ ]:
CREATE OR REPLACE TABLE fact_electric_vehicle_sales (
    sale_id  INT AUTOINCREMENT PRIMARY KEY,
    vehicle_id INT REFERENCES dim_vehicle(vehicle_id),
    city_id INT REFERENCES dim_city(city_id),
    time_id INT REFERENCES dim_time(time_id),
    sale_price INTEGER,
    electric_range INTEGER,
    cafv_eligibility STRING,
    buyer_id STRING
);


In [ ]:
INSERT INTO dim_vehicle (vin, make, model, year_model, electric_vehicle_type, base_msrp)
SELECT DISTINCT
    COALESCE("vin_110",'UNKNOWN VIN') AS vin,
    COALESCE("make",'UNKNOWN MAKE') AS make,
    COALESCE("model",'UNKNOWN MODEL') AS model,
    COALESCE("model_year", 0000) AS year_model,
    COALESCE("electric_vehicle_type", 'UNKNOWN TYPE') AS electric_vehicle_type,
    COALESCE("base_msrp", 0) AS "base_msrp"
FROM
    electricvehicles.staging.ev_population_stg;


In [ ]:
INSERT INTO dim_city (city_name, state, county, postal_code)
SELECT DISTINCT
    COALESCE("city",'UNKNOWN CITY') AS city_name,
    COALESCE("state",'UNKNOWN STATE') as state,
    COALESCE("county",'UNKNOWN COUNTY') AS county,
    COALESCE("postal_code", 'UNKNOWN ZIP') AS postal_code
FROM
    electricvehicles.staging.ev_population_stg;


In [ ]:
INSERT INTO dim_time (sale_date, sale_day, sale_month, sale_quarter, sale_year, resale_date)
SELECT DISTINCT
    COALESCE("created_at"::TIMESTAMP, '1970-01-01'::TIMESTAMP) AS sale_date,
    COALESCE(EXTRACT(DAY FROM "created_at"::TIMESTAMP), 1) AS sale_day,
    COALESCE(EXTRACT(MONTH FROM "created_at"::TIMESTAMP), 1) AS sale_month,
    COALESCE(EXTRACT(QUARTER FROM "created_at"::TIMESTAMP), 1) AS sale_quarter,
    COALESCE(EXTRACT(YEAR FROM "created_at"::TIMESTAMP), 1970) AS sale_year,
    COALESCE("updated_at"::TIMESTAMP, '1970-01-01'::TIMESTAMP) AS resale_date,
FROM
    electricvehicles.staging.ev_population_stg;


In [ ]:
INSERT INTO fact_electric_vehicle_sales (
vehicle_id,
city_id,
time_id,
sale_price,
electric_range,
CAFV_ELIGIBILITY,
buyer_id
)
SELECT DISTINCT
v.vehicle_id,
c.city_id,
t.time_id,
COALESCE(stg."base_msrp", 0) as sale_price,
COALESCE(stg."electric_range",0) as electric_range,
COALESCE(stg."clean_alternative_fuel_vehicle_cafv_eligibility",'unknown') as CAFV_ELIGIBILITY,
COALESCE(stg."id", 'unknown buyer') as buyer_id
FROM electricvehicles.staging.ev_population_stg stg
JOIN electricvehicles.staging.dim_vehicle v
ON stg."vin_110" = v.vin
JOIN electricvehicles.staging.dim_city c
ON stg."postal_code" = c.postal_code
JOIN electricvehicles.staging.dim_time t
ON stg."created_at"::TIMESTAMP = t.sale_date;

## Example queries to answer some common questions:

- Top EV's/PHEV/BEV based on electric range and model
- States/Cities with the most EV's/PHEV/BEV


In [ ]:
SELECT

v.make,
v.model,
MAX(f.electric_range) as max_range,
FROM electricvehicles.staging.fact_electric_vehicle_sales f 
LEFT JOIN electricvehicles.staging.dim_vehicle v 
ON f.vehicle_id = v.vehicle_id
-- WHERE v.electric_vehicle_type LIKE '%PHEV%'
GROUP BY 1,2
ORDER BY max_range DESC


In [ ]:
SELECT
v.make,
c.state,
COUNT(*) as totl_cars
-- APPROX_COUNT_DISTINCT(v.vin) as cars
FROM electricvehicles.staging.fact_electric_vehicle_sales f 
LEFT JOIN electricvehicles.staging.dim_vehicle v 
ON f.vehicle_id = v.vehicle_id
LEFT JOIN electricvehicles.staging.dim_city c
ON f.city_id = c.city_id
-- WHERE v.electric_vehicle_type LIKE '%PHEV%'
GROUP BY 1,2
ORDER BY  totl_cars DESC